# **The Molecular Treasure Hunt: Part 5: Saving your greatest "hits"**

*An &#8491;ngstrom sized adventure by Sarah Harris (Leeds Physics) and Geoff Wells (UCL Pharmacy)*

"*It is known that there are an infinte number of worlds, simply because there is an infinite amount of space for them to be in. However, not every one of them is inhabited.*" Douglas Adams, The Restaurant at the End of the Universe

# A few things to think about
## Dynamical considerations
In the molecular treasure hunt, we are interested in protein dynamics. We have therefore structured the output so that for each ligand you identify as a "hit", you obtain the pose of that ligand complexed with each protein conformation. It may be that some of the structures you obtain did not score well in terms of Vina_BE, but in spite of this, those complexes will be included in your dataset, because we have decided not to elimate any information about dynamics from your results. 

You may find that the ligand pose changes when the protein conformation changes. We are very interested to know how often and why this happens. We are also curious to see whether more flexible binding sites are more "promiscious" with respect to ligand pose.

# Saving our 'best' results
Now we are ready to save our 'best' hit results into new folders (defined below as 'MTH_Hits' and 'MTH_Protein_Hits'), and we use the compound selection that we made in Part 4.

In [ ]:
#This imports the python modules that we need for the notebook...
from pathlib import Path
from termcolor import colored
from glob import glob
import oddt

In [ ]:
#We load the hit ligand list that we made in Part 4
hits_file = 'hit_ligands.txt'
hits_folder = ('MTH_Hits')
protein_hits_folder = ('MTH_Protein_Hits')

#Then we make the two results folders
rf = Path(hits_folder)
try:
    rf.mkdir()
except FileExistsError as exc:
    print(colored('The hits folder is already present - check that it is empty before starting!', 'blue',  attrs=['bold']))

rf1 = Path(protein_hits_folder)
try:
    rf1.mkdir()
except FileExistsError as exc:
    print(colored('The protein hits folder is already present - check that it is empty before starting!', 'blue',  attrs=['bold']))

In [ ]:
#We find our chosen molecules in our previously saved results files and save them in new files
path_results_filenames = glob('MTH_Results/VinaResults*.mol2')
path_results_filenames.sort()
hit_ligands=[]
with open(hits_file, 'r') as f:
    for line in f:
        line=line.rsplit()
        hit_ligands.append(line)
f.close()
for path_results_filename in path_results_filenames:
    results_filename=path_results_filename.split('/')[-1]
    results_filename_prefix=results_filename.removesuffix('.mol2')
    protein=results_filename_prefix.split('_')[1]
    all_mols = list(oddt.toolkit.readfile('mol2', path_results_filename))
    hits_out_filename=hits_folder + '/' + results_filename_prefix + '_hit_ligands.mol2'
    with open(hits_out_filename, 'w') as contents:
        with open(hits_out_filename, 'a') as contents:
            for mol in all_mols:
                for hit_ligand in hit_ligands:
                    if mol.data['Name'] in hit_ligand:
                        hit=mol.write('mol2')
                        contents.write("########## Name: " + mol.data['Name'] + '\n' + 
                                       "########## Vina_BE: " + mol.data['Vina_BE'] + '\n' + 
                                       "########## MW: " + mol.data['MW'] + '\n' + 
                                       "########## clogP: " + mol.data['clogP'] + '\n' + 
                                       "########## nonHatoms: " + mol.data['nonHatoms'] + '\n' + 
                                       "########## Vina_LE: " + mol.data['Vina_LE'] + '\n' + 
                                       "########## Vina_LipE: " + mol.data['Vina_LipE'] + '\n' + 
                                       "########## Close_contacts: " + mol.data['Close_contacts'] + '\n' + 
                                       "########## Hydrophob_contacts: " + mol.data['Hydrophob_contacts'] + '\n' + 
                                       "########## HBonds: " + mol.data['HBonds'] + '\n' + 
                                       "########## Salt_Bridges: " + mol.data['Salt_Bridges'] + '\n' + 
                                       "########## Pi-Pi_Parallel: " + mol.data['Pi-Pi_Parallel'] + '\n' + 
                                       "########## Pi-Pi_Perpendicular: " + mol.data['Pi-Pi_Perpendicular'] + '\n' + 
                                       "########## Pi-Cation: " + mol.data['Pi-Cation'] + '\n' + 
                                       "########## Halogen_Bonds: " + mol.data['Halogen_Bonds'] + '\n' + 
                                       hit)
        contents.close()



### Aminating outputs for docking runs against multiple proteins that are the same (e.g. from MD)
We can run the cell below if we have docking data for several protein conformers that are the from an MD simulation. If we only have one protein or our proteins are not identical (e.g. mutants, isoforms, etc.) DO NOT run the cell below and proceed to run the final cell only.

In [ ]:
#Here we write out the files to the MTH_Protein_Hits folder
for hit_ligand in hit_ligands:
    lig=""
    for ele in hit_ligand: 
        lig += ele
        print(lig)
    
        protein_hits_out_filename=protein_hits_folder + '/' + lig + '_all_protein_poses.pdb'
        with open(protein_hits_out_filename, 'w') as contents:
            with open(protein_hits_out_filename, 'a') as contents:
                for path_results_filename in path_results_filenames:
                    results_filename=path_results_filename.split('/')[-1]
                    results_filename_prefix=results_filename.removesuffix('.mol2')
                    protein_suffix=results_filename_prefix.split('VinaResults_')[1]
                    protein=protein_suffix.split('_rescored')[0]
                    print(protein)
                    protein_file='../receptor/'+ protein + '.pdb'
                    protein_in=next(oddt.toolkit.readfile('pdb', protein_file))
                    protein_out=protein_in.write('pdb')
                    protein_number=protein.split('_')[-1]
                    all_mols = list(oddt.toolkit.readfile('mol2', path_results_filename))
                    for mol in all_mols:
                        if mol.data['Name'] in hit_ligand:
                            hit=mol.write('pdb')
                            contents.write("MODEL " + protein_number + '\n' +
                                           str(protein_out) + '\n' + 'TER' + '\n' +
                                           hit +
                                           "ENDMDL" + '\n')
            contents.close()

out_pdb_files=glob(protein_hits_folder + '/*.pdb')
for file in out_pdb_files:
    with open(file, 'r') as f:
        lines = f.readlines()
    f.close()
    with open(file, 'w') as contents:   
        for line in lines:
            if "MODEL" in line or "ATOM" in line or "HETATM" in line or "TER" in line or "ENDMDL" in line:
                contents.write(line)
    contents.close()

To visualise how your ligand pose changes as the protein conformation changes, you need to read these pdb files into Chimera. Please watch the video accompanying this notebook to see how to do this.

If you wish, you can visualise the .sdf files in either Data Warrior or Pymol (but this is not essential to your mission!).

In [ ]:
#Write sdf files for use in DataWarrior etc.
mol2_files = glob('MTH_Hits/VinaResults_*.mol2')
for mol2_file in mol2_files:
    mol2_prefix = mol2_file.removesuffix('.mol2')
    sdf_out = mol2_prefix + '.sdf'
    mols = list(oddt.toolkit.readfile('mol2', mol2_file))
    with open (sdf_out, 'w') as contents:
        for mol in mols:
            sdf = mol.write('sdf')
            with open (sdf_out, 'a') as contents:
                contents.write(sdf)
    contents.close()

# Caveats, Confessions and Future Work

## Ligand pose issues
You may remember that one of the details of the protocol in Part 1 of your treasure hunt was that for each ligand we selected only the single "best" docked conformer for each receptor structure, to reduce the complexity of our dataset. Our ligand libraries contain between 80 and ~2000 ligands and hopefully you have considered around 10 protein conformers, but if we additionally include all ligand poses from the docking we increase this by an order of magnitude. Subsequent experience has shown that when these multiple ligand conformers (for a single protein receptor) are energy minimised, the scores can swap in rank order. Although the differences in energy are small, the difference in ligand pose can be disturbingly large. We are thinking about this very carefully.

## Problems with protons
Protons generally make up around half of the total number of atoms in a biomolecular structure, but are frequently conveniently overlooked because they cannot be resolved by X-ray crystallography. This is a problem if you would like to run a molecular dynamics simulation, because these sophisticated biomolecular models explicitly require input structures that are chemically correct.

The original ligand libraries, which generally contain 2D chemical structures (e.g. when you download them from  the ZINC database, Enamine or other sources), have been lovingly inflated into 3D and protonated by Geoff (using the molconvert software https://docs.chemaxon.com/display/docs/Molecule_file_conversion_with_Molconverter.html), who can view the structure of a drug and know whether it violates fundamental chemical laws. This is not a common skill, and so as far as possible we want to automate this decision making process. The structures are named according to their reference numbers so can be easily searched (and researched) online if they are of interest!

The original .mol2 files in your ligand libraries are correctly protonated (there may still be some horrors that Geoff just hasn't seen yet in the structures - please let us know if you see an issue!). However, in Part 2, you use a pdbqt file that contains only the polar hydrogens to perform the Vina docking and minimisation steps.

Ligands and proteins contain *titratable groups* (e.g. acids and bases) which change their protonation state due to the local electrostatic environment, which here is determined by the biochemistry and solvent accessibility of the site. This means that the protonation state of your ligand may change when it binds to the protein (this is called a *pKa shift*), and may also be dependent on protein conformer, because this changes charge-charge interactions. This has not been included at the docking stage.

We propose that the treatment of protontation states is key to successful rational drug design. This is because protonation states often determine the ability of biomolecules to form hydrogen bonds and electrostatic interactions, which are vital for specificity. You will notice that Hbonds are a particular column that we use for selecting hits in Parts 3 and 4. It may also be possible to design compounds that are sufficiently lipophilic to cross a biological membrane, but which become more polar when exposed to the electrostatic environment of your protein receptor binding site through shifts in pka. 

Moreover, some very successful drugs make explicit use of changes in acidity as part of their function, and this is especially true for anti-cancer drugs where the tumour environment is more acidic than healthy tissues. 

## Complications involving the solvent (water)
Many protein-ligand structures show that specific interactions with the solvent can be important for stabilising the bound conformation of the ligand. This can be due to the formation of hydrogen bond networks in which water molecules 'bridge' gaps between the ligand and the protein. Such interactions can be missed in docking calculations, because many protocols strip the solvent and ions from the systems before running the docking calculations. However, water residues can be included in the calculations. Indeed, in one of our case studies, water residues that coordinate to a magnesium ion are included as part of the protein pdb file.

## Difficulties with induced fit
When we measure binding events in experiments we "record" an ensemble of strong/durable and weak/transient binding events that contribute to our measurement. In contrast, during docking studies we focus on the *best* docked conformations that we can determine. This is important because it directs us towards the highest affinity interaction that a ligand could form, but it doesn't account for the *ensemble* of ways and timescales over which a ligand could bind to a protein. This is partly why disappointments are frequent in computational docking studies! Hang in there - it could be a bumpy ride because free energy landscapes contain multiple minima!

One of the most elegant aspects of molecular recognition is that the protein receptor target undergoes a conformational change when the ligand binds. This is because proteins are "soft matter" physical systems, which can explore multiple free energy minima at an ambient temperature (300 K). This provides an exquiste "fine tuning" of biomolecular interactions that is (arguably) one of the key thermodynamics signatures of life. It is impossible to consider these effects unless we allow the receptor to adapt to our ligand pose. The Vina software is capable of performing a flexible dock where you can specify the residues that are mobile. Ideally, you would do this using distance based criteria from the ligand, but in practise this is so slow that manual intervention is required to reduce the number of flexible residues. We have therefore not attempted this in the workflow here. 

Our current approach partially considers protein receptor conformational flexibility by considering multiple protein conformations. However, protein conformational changes due to thermal fluctuations are inherently multi-scale. This means that over short (e.g. ps-ns) timescales changes in protein and ligand structure involve merely local rotations in bond and dihedral angles. Over longer (e.g $\mu$s) timescales, flexible proteins may undergo global conformational changes in shape that modify the binding site. As yet, we have a poor understanding of the relative importance of local versus global conformational changes, or the timescales over which these occur. We hope that by using computation these notebooks will address some of these fundamental biochemical-physical questions. 

Please note that this question is not independent of the issue with protonation, as you may have guessed!!

**Future work:** If the presence of the ligand modifies the conformational landscape that the protein explores, we need a physics-based (e.g. molecular dynamics forcefield) to predict the resulting protein-ligand complexes and their binding energies. One method to assess the importance of induced fit, certainly at the local (e.g. side-chain) level, is to perform a molecular dynamics simulation of your receptor-ligand complex. In our experience, these simulations based on docking poses often result in ligands diffusing out of the binding pocket. We are very interested to know whether the quality of the structure based on docking score correlates with the stability of a complex subjected to molecular dynamics. We are confident that much of this workflow can be extended to automate the simulation construction and launching process.    

"*Little did I know where the chief peril lay! ... Torment in the dark was the danger that I feared, and it did not hold me back. But I would have never come, had I known the danger of light and joy.*" J.R.R. Tolkien, The Fellowship of the Ring.

Good luck!!

Sarah and Geoff

(an "out-of-office studios" $O^{3}S$ production)